In [1]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
from src import matcher as M
from shapely.geometry import Point

In [ ]:
G = ox.load_graphml(
    '../processed_data/bkk_tmc_graph.graphml'
)

In [ ]:
G_proj = ox.project_graph(G)
nodes, edges = ox.graph_to_gdfs(G_proj)

depot_df = pd.read_csv('../processed_data/depot_locations.csv')
depot_df["geometry"] = depot_df.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)

depot_df["edge_id"], depot_df["dist_to_candidate"] =ox.distance.nearest_edges(G, depot_df.lon, depot_df.lat, return_dist=True)

depot_gdf = gpd.GeoDataFrame(
    depot_df,
    geometry=depot_df.geometry
).set_crs(4326).to_crs(32647)

proj_points = []
edge_geoms = []
for _, point in depot_gdf.iterrows():
    point_geom = point.geometry
    edge_id = point.edge_id
    u, v, key = edge_id
    edge_geom = edges.loc[u, v, key].geometry
    s = edge_geom.project(point_geom)
    projected_point = edge_geom.interpolate(s)
    proj_points.append(projected_point)
    edge_geoms.append(edge_geom)

depot_gdf["projected_point"] = proj_points
depot_gdf["edge_geom"] = edge_geoms

depots = {}
for loc_code, sub in depot_gdf.groupby('name:en'):
    depots[loc_code] = {
        "point_geometry": sub["geometry"].tolist(),
        "projected_point": sub["projected_point"].tolist(),
        "edge_geometry": sub["edge_geom"].tolist(),
        "edge_id": sub["edge_id"].tolist(),
        "dist_to_candidate": sub["dist_to_candidate"].tolist(),
        "point_id": [""]         # 0..K-1
    }

G_depot = M.build_augmented_graph_batch(G_proj, edges.reset_index(), depots, vtype='hosp')

G_depot = ox.project_graph(G_depot, to_crs="EPSG:4326")

In [ ]:
G_proj = ox.project_graph(G_depot)
nodes, edges = ox.graph_to_gdfs(G_proj)

accident_df = pd.read_csv('../processed_data/cluster_centroids_stats.csv')
accident_df["geometry"] = accident_df.apply(lambda row: Point(row["Centroid_Longitude"], row["Centroid_Latitude"]), axis=1)

accident_df["edge_id"], accident_df["dist_to_candidate"] =ox.distance.nearest_edges(G_depot, accident_df.Centroid_Longitude, accident_df.Centroid_Latitude, return_dist=True)

accident_gdf = gpd.GeoDataFrame(
    accident_df,
    geometry=accident_df.geometry
).set_crs(4326).to_crs(32647)

proj_points = []
edge_geoms = []
for _, point in accident_gdf.iterrows():
    point_geom = point.geometry
    edge_id = point.edge_id
    u, v, key = edge_id
    edge_geom = edges.loc[u, v, key].geometry
    s = edge_geom.project(point_geom)
    projected_point = edge_geom.interpolate(s)
    proj_points.append(projected_point)
    edge_geoms.append(edge_geom)

accident_gdf["projected_point"] = proj_points
accident_gdf["edge_geom"] = edge_geoms

accidents = {}
for loc_code, sub in accident_gdf.groupby('Cluster_ID'):
    accidents[loc_code] = {
        "point_geometry": sub["geometry"].tolist(),
        "projected_point": sub["projected_point"].tolist(),
        "edge_geometry": sub["edge_geom"].tolist(),
        "edge_id": sub["edge_id"].tolist(),
        "dist_to_candidate": sub["dist_to_candidate"].tolist(),
        "point_id": [""]         # 0..K-1
    }
    
G_final = M.build_augmented_graph_batch(G_proj, edges.reset_index(), accidents, vtype='accident')

In [5]:
import folium

def plot_graph_folium_with_virtual_nodes(G, label_nodes=False, zoom_start=14):
    """
    Plot graph G on Folium, highlighting virtual nodes (names starting with "virt").
    
    - Virtual nodes: big red markers
    - Normal nodes: small blue markers
    - Edges: gray polylines
    """

    # -------------------------------
    # 1. Determine center of the map
    # -------------------------------
    # Use the mean of all node coords
    xs = [data["x"] for _, data in G.nodes(data=True)]
    ys = [data["y"] for _, data in G.nodes(data=True)]
    center_lat = sum(ys) / len(ys)
    center_lon = sum(xs) / len(xs)

    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start)

    # -------------------------------
    # 2. Plot edges
    # -------------------------------

    nodes, edges = ox.graph_to_gdfs(G)
    for _, edge in edges.iterrows():
        geom = edge["geometry"]
        coords = [(lat, lon) for lon, lat in geom.coords]  # flip order
        folium.PolyLine(coords, color="gray", weight=2, opacity=1).add_to(m)

    # -------------------------------
    # 3. Plot nodes
    # -------------------------------
    for osmid, node in nodes.iterrows():
        mgeom = node["geometry"]
        if node["vtype"] == "hosp":
            folium.CircleMarker(
                location=[mgeom.y, mgeom.x],
                radius=7,
                color="green",
                fill=True,
                fill_color="green",
                fill_opacity=1,
                popup=str(osmid)
            ).add_to(m)
        
            if label_nodes:
                folium.map.Marker(
                    [mgeom.y, mgeom.x],
                    icon=folium.DivIcon(
                        html=f"""<div style="font-size:10pt; color:black">{osmid}</div>"""
                    )
                ).add_to(m)

        elif node["vtype"] == "accident":
            folium.CircleMarker(
                location=[mgeom.y, mgeom.x],
                radius=7,
                color="red",
                fill=True,
                fill_color="red",
                fill_opacity=1,
                popup=str(osmid)
            ).add_to(m)
        
            if label_nodes:
                folium.map.Marker(
                    [mgeom.y, mgeom.x],
                    icon=folium.DivIcon(
                        html=f"""<div style="font-size:10pt; color:black">{osmid}</div>"""
                    )
                ).add_to(m)

        else:
            folium.CircleMarker(
                location=[mgeom.y, mgeom.x],
                radius=1,
                color="gray",
                fill=True,
                fill_color="gray",
                fill_opacity=1
            ).add_to(m)

    return m


In [ ]:
m = plot_graph_folium_with_virtual_nodes(ox.project_graph(G_final, to_crs="EPSG:4326"), label_nodes=True)

m